<a href="https://colab.research.google.com/github/abhik-99/MFSGC/blob/master/Univariate_and_Multivariate_Filter_Methods_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing of Filter Methods (Univariate)

The First Part of this notebook evaluates how a single filtering method performs against **KNN, SVM, Decision Tree and Naive Bayes Algorithm** as well as **Ensemble**.\
At each step, a particular filtering method is chosen and then it is evaluated.\
\
In the second part, multiple filtering methods are taken for scoring the features. Top *n* features from each filtered dataset is taken and performance is analysed against the same classification algorithms.\
\
**Leave One Out Cross Validation (LOOCV)** is used.

## General Steps before evaluation
The cells below are same for univariate and multivariate filter method evaluation. As such, they may be executed only once.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from sklearn.feature_selection import chi2
!pip install -U -q PyDrive
!pip install skfeature-chappers

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':'1oaOATE0D_f8MGPIMJOMXYVt0hBUWNCKV'}) # replace the id with id of file you want to access
# For Leukemia- 1xcL-LT-E_gUqWLlqqeVJP1DVHHpiAGe_
# For Colon - 1AUOto0GhTHW9fX52XSsf9kzYJS5ggv0G
# for Prostate - 13Hf7uGbyJ1sWYo8KDRDL8scm-2Fs9_gd
# For Lung- 1xuLzTWDGUbr4x3Pq1dnJj08MZqBB5I3U
# for Rahc - 1oaOATE0D_f8MGPIMJOMXYVt0hBUWNCKV
# for Raoa - 1d2vhPcT3I7ZFcAGOQYVLGB3Jx_vEMata
# for Rbreast - 1Vf-h8zfVP_twMXivcJJtbWtjThShUHvn
# for SRBCT - 1rO5EEvsoRJl2VVUB3ywKUd3kNiQ24oy3
# for MLL - 1rS7x4x_DhrUzaBhrgKMQH3uIaLJdPgW3
# for Breast - 1enhhyA4u2ByvOjnF81WoHflVNpXtfKpu
downloaded.GetContentFile('data.txt')

In [ ]:
#DATASET is the name of the dataset being used.
DATASET="RAHC"

#NEIGHBOURS determines neighbours arg for ReliefF
#for any dataset which contains any class sample 
# <10, make it less than 10. Eg of such dataset - SRBCT
NEIGHBOURS = 3 

#p is the number of top genes taken after sorting the filter scores
p = 5

#uncomment the line below if using the dataset splitter else leave it commented 
#data_df = pd.read_csv("%s_train.csv"%(DATASET),index_col=0)

#uncomment the lines below if using the original dataset
dataset = pd.read_table("data.txt",header=None)
data_df = dataset



target = data_df.iloc[:,-1]
feature = pd.DataFrame(data_df.iloc[:,:-1].values,dtype='float')
m,n = feature.shape
print(m,n)
print(feature.head())
print("Number of classes - ")
classes = np.unique(target)
for x in classes:
  print("Class -",x,"Number of Sampples -", len(np.where(target == x)[0]))

feature_norm=pd.DataFrame(MinMaxScaler().fit_transform(feature))

In [ ]:
#construction of ReliefF function

"""
Given a dataset, number of random instances to pick form the dataset and
number of features to consider in each iteration (k), the function returns the weigths of the attributes
of the dataset.
These weigths can then be used as the final results out of the ReliefF algorithm

Paper-

Marko Robnik-ˇSikonja and Igor Kononenko. Theoretical and empirical analysis of relieff
and rrelieff. Machine learning, 53(1-2):23–69, 2003.

"""
def reliefF(feature,targetClass,n_neighbours=10,instances_to_select=10):
  import numpy as np
  
  #feature=df.iloc[:,:-1]
  #targetClass=df.iloc[:,-1]
  m,n=feature.shape
  #print("Dimensions:",m,n)
  weights=np.zeros(n,dtype='int')

  classes=np.unique(targetClass)

  m2=instances_to_select #number of features to pickup randomly from the 
  k=n_neighbours #number of neighbours to consider

  instances=np.array(list(range(1,m)))
  

  minimums=np.min(feature.values,axis=0)
  maximums=np.max(feature.values,axis=0)

  differ=np.subtract(maximums,minimums)


  for i in range(m2):
    chosen=np.random.choice(instances[:-1])
    instances=np.delete(instances,np.where(instances==chosen))

    rI=feature.iloc[chosen,:].values

    instanceClass=targetClass[chosen]
    probIClass=len(np.where(targetClass==instanceClass)[0])/m #getting the probaility of choosing this class

    hit=[]
    miss={}

    low,high,tem1,tem2=0,m,(chosen-1),(chosen+1)
    hitFlag=True
    missFlag=True
    
    
    while(hitFlag==True):

      if targetClass[tem1]==instanceClass:
        hit.append(tem1)
      if targetClass[tem2]==instanceClass:
        hit.append(tem2)
      if len(hit)==k:
        hitFlag=False
      if tem1>0:
        tem1-=1
      if tem2<m-1:
        tem2+=1
    for x in classes:
      if x==instanceClass:
        continue
      #print(instanceClass,x)
      cli=max(np.where(targetClass==x)[0]) #finding the last instance of the class x
      if cli>chosen:
        tem=min(np.where(targetClass==x)[0])
        miss[x]=list(range(tem,tem+k))
      else:
        tem=max(np.where(targetClass==x)[0])
        miss[x]=list(range(cli-k+1,cli))

    #print("Chosen-",chosen,"Hits-",hit,"Misses-",miss)

    totalHit=np.zeros(n,dtype='int')

    for hit in range(k):
      hI=feature.iloc[hit,:].values
      dRH=np.divide(np.abs(np.subtract(rI,hI)),differ)
      dRH=dRH/(m2*k)
      totalHit=np.add(totalHit,dRH)

    totalMiss=np.zeros(n,dtype='int')

    for eachClass in miss:

      tMiss=np.zeros(n,dtype='int')
      pclass=len(np.where(targetClass==eachClass)[0])/m #getting the probability of getting this class
      postProb=pclass/(1-probIClass) #calculating the posterior probanility of getting this class

      for eachMiss in miss[eachClass]:
        mI=feature.iloc[eachMiss,:].values
        dRM=np.divide(np.abs(np.subtract(rI,mI)),differ)
        dRM=dRM/(m2*k)
        tMiss=np.add(tMiss,dRM)

      totalMiss=np.add(totalMiss,(tMiss*postProb))

    weights=np.add(weights,totalMiss)
    weights=np.subtract(weights,totalHit) 
    
  return weights

In [ ]:
#This function discretizes the given features into 3 categories
def discretize_feature(feature):
  import numpy as np
  mean=np.mean(feature)
  std=np.std(feature)
  discretized=np.copy(feature)
  
  discretized[np.where(feature<(mean+std/2)) ,]=2#within 1/2 std div
  discretized[np.where(feature>(mean-std/2)),]=2#within 1/2 std div
  
  discretized[np.where(feature>(mean+std/2)),]=0#greater than half
  discretized[np.where(feature<(mean-std/2)),]=1#less than half
  
  return discretized

def Xfreq(x):
  xL={}
  for e in x:
    if e not in xL:
      xL[e]=0
    else:
      xL[e]+=1
  for e in xL:
    xL[e]/=len(x)
  return xL

def XYfreq(x,y):
  freq={}
  import numpy as np
  
  rX=np.unique(x)
  rY=np.unique(y)
      
  for e in rX:
    for f in rY:
      freq[(e,f)]=round(len(np.where(y[np.where(x==e)[0]]==f)[0])/len(x),4)
       
  return freq

def mutual_info(x,y):
  import numpy as np
  
  xFreq=Xfreq(x)
  yFreq=Xfreq(y)
  joint=XYfreq(x,y)
  
  Xentropy=0
  for e in xFreq:
    if xFreq[e]!=0:
      Xentropy-=xFreq[e]*np.log2(xFreq[e])
      
  Yentropy=0
  for e in yFreq:
    if yFreq[e]!=0:
      Yentropy-=yFreq[e]*np.log2(yFreq[e])
      
  jentropy=0
  for e in xFreq:
    for f in yFreq:
      if joint[(e,f)]!=0:
        jentropy-=joint[(e,f)]*np.log2(joint[(e,f)])
  
  return (Xentropy+Yentropy-jentropy)

def mutual_info_wrapper(features,targetClass):
  import numpy as np
  mi=np.array([])
  for x in features:
    discrete=discretize_feature(features[x])
    mi=np.append(mi,mutual_info(discrete,targetClass))
  return np.array(mi)

In [ ]:
"""
This cell is used for getting the t-test scores
"""

def t_test(df,target):
  """
  Input:
  df= Dataframe of features (n_samples,n_features)
  target= Pandas Series/1D Numpy Array containing the class labels (n_samples)
  
  Output:
  scores= Descendingly Sorted array of features based on t-test 
  """
  import numpy as np
  from scipy.stats import ttest_ind
  scores=ttest_ind(df[:][targetClass==0],df[:][targetClass==1])[0] #Storing just the t-test scores and discarding the p-values from the result.
  
  scores=np.argsort(scores,0)
  
  return scores[::-1]

  

In [ ]:
from scipy.sparse import *
def fisher_score(X, y):
    import numpy as np
    
    from skfeature.utility.construct_W import construct_W
    """
    This function implements the fisher score feature selection, steps are as follows:
    1. Construct the affinity matrix W in fisher score way
    2. For the r-th feature, we define fr = X(:,r), D = diag(W*ones), ones = [1,...,1]', L = D - W
    3. Let fr_hat = fr - (fr'*D*ones)*ones/(ones'*D*ones)
    4. Fisher score for the r-th feature is score = (fr_hat'*D*fr_hat)/(fr_hat'*L*fr_hat)-1

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    y: {numpy array}, shape (n_samples,)
        input class labels

    Output
    ------
    score: {numpy array}, shape (n_features,)
        fisher score for each feature

    Reference
    ---------
    He, Xiaofei et al. "Laplacian Score for Feature Selection." NIPS 2005.
    Duda, Richard et al. "Pattern classification." John Wiley & Sons, 2012.
    """

    # Construct weight matrix W in a fisherScore way
    kwargs = {"neighbor_mode": "supervised", "fisher_score": True, 'y': y}
    W = construct_W(X, **kwargs)

    # build the diagonal D matrix from affinity matrix W
    D = np.array(W.sum(axis=1))
    L = W
    tmp = np.dot(np.transpose(D), X)
    D = diags(np.transpose(D), [0])
    Xt = np.transpose(X)
    t1 = np.transpose(np.dot(Xt, D.todense()))
    t2 = np.transpose(np.dot(Xt, L.todense()))
    # compute the numerator of Lr
    D_prime = np.sum(np.multiply(t1, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # compute the denominator of Lr
    L_prime = np.sum(np.multiply(t2, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # avoid the denominator of Lr to be 0
    D_prime[D_prime < 1e-12] = 10000
    lap_score = 1 - np.array(np.multiply(L_prime, 1/D_prime))[0, :]

    # compute fisher score from laplacian score, where fisher_score = 1/lap_score - 1
    score = 1.0/lap_score - 1
    return np.transpose(score)

In [ ]:
#signal to noise ratio
#using weighted one-vs-all strategy for multi-class data
def signaltonoise(feature, target, axis = 0, ddof = 0):
  import numpy as np
  classes = np.unique(target)
  if len(feature.shape)<2:
    feature = feature.reshape(-1,1)
  row, _ = feature.shape
  if len(classes) <= 2:
    m = None
    std = 0
    for each in classes:
      idx = np.where(target == each)[0]
      #convinient way of doing m1-m2
      if m is None:
        m = feature.iloc[idx, :].mean(axis)
      else:
        m -= feature.iloc[idx, :].mean(axis)

      #sd1+sd2
      std += feature.iloc[idx, :].std(axis = axis, ddof = ddof)

    return np.asanyarray(m/std)

  else:
    snr_scores = [] #for storing the weighted scores
    #using the one vs all strategy for each class with
    for each in classes:
      idx = np.where(target == each)[0]
      idxn = np.where(target != each)[0]
      m = feature.iloc[idx, :].mean(axis) - feature.iloc[idxn, :].mean(axis)
      std = feature.iloc[idx, :].std(axis = axis, ddof = ddof) + feature.iloc[idxn, :].std(axis = axis, ddof = ddof) 
      snr_scores.append((m/std) * len(idx)/row) #weighted snr

    return np.asanyarray(snr_scores).sum(axis = axis)

In [ ]:
#Gini index, forked from oliviaguest from GitHub.

def gini(array):
    import numpy as np
    """Calculate the Gini coefficient of a numpy array."""
    # based on bottom eq:
    # http://www.statsdirect.com/help/generatedimages/equations/equation154.svg
    # from:
    # http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    # All values are treated equally, arrays must be 1d:
    array = array.flatten()
    if np.amin(array) < 0:
        # Values cannot be negative:
        array -= np.amin(array)
    # Values cannot be 0:
    array += 0.0000001
    # Values must be sorted:
    array = np.sort(array)
    # Index per array element:
    index = np.arange(1,array.shape[0]+1)
    # Number of array elements:
    n = array.shape[0]
    # Gini coefficient:
    return ((np.sum((2 * index - n  - 1) * array)) / (n * np.sum(array)))
def gini_wrapper(features):
  return np.argsort([gini(features[x].values) for x in range(features.shape[1])],0).tolist()[::-1]


In [ ]:
def pearson_corr(feature,targetClass):
  import numpy as np
  coef=[np.abs(np.corrcoef(feature[i].values,targetClass)[0,1]) for i in range(feature.shape[1])]
  coef=[0 if np.isnan(i) else i for i in coef]
  return np.argsort(coef,0).tolist()[::-1]


In [ ]:

def feature_ranking(score):
    """
    Rank features in descending order according to fisher score, the larger the fisher score, the more important the
    feature is
    """
    idx = np.argsort(score, 0)
    return idx[::-1]

In [ ]:

relief_score=reliefF(feature,target,NEIGHBOURS)

mutual_inf=mutual_info_wrapper(feature,target)

mms=MinMaxScaler()
nfeature=mms.fit_transform(feature)
chi_score,p_val=chi2(nfeature,target)

p_corr = pearson_corr(feature, target)

f_score = fisher_score(feature.values, target)

tt_score = t_test(feature, target)

snr_score = signaltonoise(feature, target)

In [ ]:
#The Features are sorted as per their scores
sorted_relief = feature_ranking(reliefF(feature,target,NEIGHBOURS))[:p]

sorted_mi = feature_ranking(mutual_info_wrapper(feature,target))[:p]

mms=MinMaxScaler()
nfeature=mms.fit_transform(feature)
chi_score,_=chi2(nfeature,target)
sorted_chi = feature_ranking(chi_score)[:p]

sorted_pc = feature_ranking(pearson_corr(feature, target))[:p]
sorted_fs = feature_ranking(fisher_score(feature.values, target))[:p]
sorted_tt = feature_ranking(t_test(feature, target))[:p]
sorted_snr = feature_ranking(signaltonoise(feature, target))[:p]

In [ ]:
LOOCV=LeaveOneOut()
data_KNN=KNeighborsClassifier(n_neighbors= int(feature.shape[0] ** 0.5))
data_SVM=SVC(kernel='rbf',gamma='scale')
data_NB=GaussianNB()
data_Tree= DecisionTreeClassifier()
rows=feature.shape[0]
classifiers=["NB","KNN","Tree","SVM"]

keys_list=[sorted_mi_keys, sorted_relief_keys, sorted_chi_keys, sorted_pc_keys, 
           sorted_fs_keys, sorted_tt_keys, sorted_snr_keys]
cluster_list=[gene_repre_1, gene_repre_2, gene_repre_3, gene_repre_4, 
              gene_repre_5, gene_repre_6, gene_repre_7]

## **PART 1**
Univariate Filter Method Evaluation

In [ ]:
acc_matrix = pd.DataFrame()
for i in range(1,6):
  """
  Make a dataframe out of i keys from each gene_representatives from their 
  respective sorted keys. 
  Than use LOOCV to measure accuracy on Train Dataset.
  """
  cluster_df=pd.DataFrame()
  s=0
  for x in range(len(cluster_list)):
    tem_df = cluster_list[x][keys_list[x][:i]]
    for x in range(i):
      cluster_df[s] = tem_df.iloc[:,x]
      s+=1
  print(cluster_df.shape)
  acc=0
  individual_acc = np.zeros(4)
  
  for train_index,test_index in LOOCV.split(cluster_df):
    """
    Data is divided into train-test splits and then polling method is used 
    to find the classification results (ensemble of KNN,SVM,NB,Decision Tree)
    """
    train_data,train_labels=cluster_df.iloc[train_index,:],target[train_index]
    test_data,test_labels=cluster_df.iloc[test_index,:],target[test_index].values.tolist()[0]
    data_KNN.fit(train_data,train_labels)
    data_SVM.fit(train_data,train_labels)
    data_NB.fit(train_data,train_labels)
    data_Tree.fit(train_data,train_labels)

    class_list = [data_NB, data_KNN, data_Tree, data_SVM]
    results=[]
    for x in range(4):
      tem_result = class_list[x].predict(test_data)[0]
      if tem_result == test_labels:
        individual_acc[x]+=1
      results.append(tem_result)
    polling_result=0
    max_freq=0
    for x in results:
      freq=results.count(x)
      if freq>max_freq:
        max_freq=freq
        polling_result=x
    if polling_result == test_labels:
      acc+=1
  individual_acc = np.round(individual_acc/cluster_df.shape[0],4)
  individual_acc = np.append(individual_acc, np.round(acc/cluster_df.shape[0],4))
  print(individual_acc)
  acc_matrix[i] = individual_acc
acc_matrix = acc_matrix.T
acc_matrix.columns = classifiers[:]+['Ensemble']

In [ ]:
print(acc_matrix)
acc_matrix.to_csv("%s_p%sAccuracy_Matrix.csv"%(DATASET, p))

## **PART 2**
Multivariate Filter Method Evaluation